# 上下文管理器和else块

In [ ]:
# 要讨论的特性
# with 语句和上下文管理器
# for、while 和 try语句的else子句

In [ ]:
# else 子句不仅能在if语句中使用，还能在for、while和try语句中使用
# for
# 仅当for循环运行完毕时（即for循环没有被break语句终止)才运行else模块
# while
# 仅当while循环因为条件为假值而退出时（即while循环没有被break语句终止)才运行else模块
# try
# 仅当try块中没有抛出异常时才运行else块。else子句抛出异常不会由前面的except子句处理。

In [1]:
my_list = []
for item in my_list:
    if item == 'banana':
        break
else:
    raise ValueError('No banana flavor found!')

ValueError: No banana flavor found!

In [ ]:
# 上下文管理器和with块
# 上下文管理器协议包含__enter__和__exit__两个方法，不管控制流以哪种方式退出with块，都会在上下文管理器对象调用__exit__方法。

In [ ]:
# 把文件对象当上下文管理器使用
with open('mirror.py') as fp:
    src = fp.read(60)

In [2]:
class LookingGlass:
    def __enter__(self):
        import sys
        self.original_write = sys.stdout.write
        sys.stdout.write = self.reverse_write
        return 'JABBERWOCKY'

    def reverse_write(self, text):
        self.original_write(text[::-1])

    def __exit__(self, exc_type, exc_value, traceback):
        # exc_type 异常类 （例如 ZeroDivisionError）
        # exc_value 异常实例。有时会有参数传给异常构造方法，例如错误消息，这些参数可以使用exc_value.args获取
        # traceback
        import sys
        sys.stdout.write = self.original_write
        if exc_type is ZeroDivisionError:
            print('Please DO NOT divide by zero!')
            return True

In [3]:
with LookingGlass() as what:
    print ('Alice, Kitty and Snowdrop')
    print (what)
print (what)
print ('Back to normal.')

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ
JABBERWOCKY
Back to normal.


In [ ]:
# contextlib模块中的实用工具
# closing
# 如果对象提供了close()方法，但没有实现__enter__/__exit__协议，那么可以使用这个函数构建上下文管理器
# suppress
# 构建临时忽略指定异常的上下文管理器
# @contextmanager
# 这个装饰器把简单的生成器函数变成上下文管理器，这样就不用创建类去实现管理器协议了。
# ContextDecorator
# 这个是基类，用于定义基于类的上下文管理器。这种上下文管理器也能用于装饰函数，在受管理的上下文中运行整个函数。
# ExitStack

In [1]:
import contextlib
# 这样写有缺陷
@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write
    def reverse_write(text):
        original_write(text[::-1])
    sys.stdout.write = reverse_write
    # 会在yield抛出异常，但是没有处理异常，sys.stdout.write永远无法恢复
    yield 'JABBERWOCKY'
    sys.stdout.write = original_write

In [2]:
with looking_glass() as what:
    print ('Alice, Kitty and Snowdrop')
    print (what)
print (what)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ
JABBERWOCKY


In [ ]:
import contextlib

@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write
    def reverse_write(text):
        original_write(text[::-1])
    sys.stdout.write = reverse_write
    # yield语句放在try/finally语句中
    try:
        yield 'JABBERWOCKY'
    except ZeroDivisionError:
        msg = 'Please DO NOT divide by zero'
    finally:
        sys.stdout.write = original_write
        if msg:
            print (msg)
    sys.stdout.write = original_write